#Facial Expression Recognition




## Extract and Check Dataset

In [ ]:
import tarfile 

file_path = #.tar file path
tar_file = tarfile.open(file_path, 'r')
tar_file.extractall('/content/')

In [ ]:
import os
images = os.listdir('train images path')
annots = os.listdir('train annotations path')

In [ ]:
# check random images
import matplotlib.pyplot as plt
import random
import numpy as np

rand_index = random.randint(0, len(images))
img = plt.imread('train_set/images/'+images[rand_index])
plt.imshow(img)

##Lets format the data 

In [ ]:
import tensorflow as tf
import numpy as np
import os

# # define the image and annotation directories
image_dir = '/content/train_set/images'
annotation_dir = '/content/train_set/annotations'

# #for test set
# image_dir = '/content/val_set/images'
# annotation_dir = '/content/val_set/annotations'



# create a list of image filenames
image_filenames = os.listdir(image_dir)

# create a dictionary to store the annotations
annotations = {}

# iterate over the image filenames and load the corresponding annotations
for image_filename in image_filenames:
    # extract the image ID from the filename
    image_id = os.path.splitext(image_filename)[0]
    
    # load the expression, arousal, and valence annotations for the image
    exp_filename = os.path.join(annotation_dir, f'{image_id}_exp.npy')
    aro_filename = os.path.join(annotation_dir, f'{image_id}_aro.npy')
    val_filename = os.path.join(annotation_dir, f'{image_id}_val.npy')
    exp = np.load(exp_filename)
    aro = np.load(aro_filename)
    val = np.load(val_filename)
    
    # store the annotations in the dictionary
    annotations[image_id] = (exp, aro, val)
    
# create a list of image paths and their corresponding annotations
image_paths = [os.path.join(image_dir, image_filename) for image_filename in image_filenames]
annotations_list = [annotations[os.path.splitext(image_filename)[0]] for image_filename in image_filenames]

In [ ]:
# create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((image_paths, annotations_list))

In [ ]:
# define a function to preprocess the images and annotations
def preprocess_image(image_path, annotation):
    # load the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # preprocess the annotations
    exp = tf.one_hot(int(annotation[0]), depth=8)
    aro = float(annotation[1])
    val = float(annotation[2])
    
    return image, {'category_output': exp, 'aro_out': aro, 'val_out': val}

# apply the preprocessing function to the dataset
dataset = dataset.map(preprocess_image)

# split the dataset into training and validation sets 
train_dataset = dataset.take(200000)
val_dataset = dataset.skip(200000)

#for test set
# test_set = dataset

# define batch size and shuffle buffer size
batch_size = 32
shuffle_buffer_size = 10000

# shuffle and batch the training dataset
train_dataset = train_dataset.shuffle(buffer_size=shuffle_buffer_size)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)





# batch the validation dataset
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


# for test set
# test_dataset = test_set.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
train_dataset, val_dataset

##Set up Mixed Percision training

It utilizes the combo of float16 AND float32 to increase the speed


In [ ]:
#Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

##Create Feature Extraction Model

In [ ]:
#build the model
from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessig #if rescaling is needed

#base model
input_shape=(224,224,3)
base_model= tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False
#input layer
inputs= tf.keras.layers.Input(shape=input_shape, name="input_layer")
#Note: EfficientNetBX has resscaling built in if not 
# x = preprocessing.rescaling(1./255)(x)

x= base_model(inputs, training=False)

x= layers.GlobalAveragePooling2D()(x)
category_out=layers.Dense(8, activation='softmax', name='category_output')(x)
arousal_out=layers.Dense(1, activation='relu', name='aro_out')(x)
valance_out = layers.Dense(1, activation='relu', name='val_out')(x)

outputs= [category_out, arousal_out, valance_out]
model = tf.keras.Model(inputs, outputs)

#compilation
model.compile(optimizer=tf.keras.optimizers.Adam(), loss={'category_output': 'categorical_crossentropy', 'aro_out': 'mse', 'val_out': 'mse'},
              metrics={'category_output': 'accuracy', 'aro_out': 'mae', 'val_out': 'mae'})


In [ ]:
model.summary()

##Fit the model:

In [ ]:
history_feature_extraction=model.fit(train_dataset,
          steps_per_epoch=int(0.2*len(train_dataset)), #do not have compute resources
          epochs=5,
          validation_data=val_dataset,\
          validation_steps = int(0.2*len(val_dataset)),
          callbacks=[model_checkpoint])

In [ ]:
len(test_dataset)

### Evaluation of EfficientNetB0

In [ ]:
results_feature_extract=model.evaluate(test_dataset)

### Prediction with EfficientNetB0

In [ ]:
# check random images
import matplotlib.pyplot as plt
import random
import numpy as np

images = os.listdir('/content/val_set/images')
annots = os.listdir('/content/val_set/annotations')

rand_index = random.randint(0, len(images))
exp_filename = images[rand_index].replace('.jpg', '_exp.npy')
aro_filename = images[rand_index].replace('.jpg', '_aro.npy')
val_filename = images[rand_index].replace('.jpg', '_val.npy')
exp = np.load('/content/val_set/annotations/'+exp_filename)
aro = np.load('/content/val_set/annotations/'+aro_filename)
val = np.load('/content/val_set/annotations/'+val_filename)
img = plt.imread('val_set/images/'+images[rand_index])
preds = model.predict(np.expand_dims(img, 0))
plt.imshow(img)
plt.title(f'Orignal\tExp {exp}, Aro {aro}, Val {val}\nPredicted\tExp {np.argmax(preds[0])}, Aro {preds[1]}, Val {preds[2]}')

In [ ]:
a=model.predict(np.expand_dims(img, 0))
np.argmax(a[0])

## With ResNet BaseLine

In [ ]:
#build the model
from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessig #if rescaling is needed

#base model
input_shape=(224,224,3)
base_model= tf.keras.applications.resnet50.ResNet50(include_top=False)
base_model.trainable=False
#input layer
inputs= tf.keras.layers.Input(shape=input_shape, name="input_layer")

x= base_model(inputs, training=False)

x= layers.GlobalAveragePooling2D()(x)
category_out=layers.Dense(8, activation='softmax',name='category_output')(x)
arousal_out=layers.Dense(1, activation='relu', name='aro_out')(x)
valance_out = layers.Dense(1, activation='relu', name='val_out')(x)

outputs= [category_out, arousal_out, valance_out]
model_res = tf.keras.Model(inputs, outputs)

#compilation
model_res.compile(optimizer=tf.keras.optimizers.Adam(), loss={'category_output': 'categorical_crossentropy', 'aro_out': 'mse', 'val_out': 'mse'},
              metrics={'category_output': 'accuracy', 'aro_out': 'mae', 'val_out': 'mae'})

In [ ]:
model_res.summary()

In [ ]:
history_resnet=model_res.fit(train_dataset,
          steps_per_epoch=int(0.2*len(train_dataset)), 
          epochs=5,
          validation_data=val_dataset,
          validation_steps = int(0.2*len(val_dataset))
          )

### Evaluation of ResNet

In [ ]:
results_feature_extract=model_res.evaluate(test_dataset)

### Making Predictions with ResNet

In [ ]:
# check random images
import matplotlib.pyplot as plt
import random
import numpy as np

images = os.listdir('/content/val_set/images')
annots = os.listdir('/content/val_set/annotations')

rand_index = random.randint(0, len(images))
exp_filename = images[rand_index].replace('.jpg', '_exp.npy')
aro_filename = images[rand_index].replace('.jpg', '_aro.npy')
val_filename = images[rand_index].replace('.jpg', '_val.npy')
exp = np.load('/content/val_set/annotations/'+exp_filename)
aro = np.load('/content/val_set/annotations/'+aro_filename)
val = np.load('/content/val_set/annotations/'+val_filename)
img = plt.imread('val_set/images/'+images[rand_index])
preds = model_res.predict(np.expand_dims(img, 0))
plt.imshow(img)
plt.title(f'Orignal\tExp {exp}, Aro {aro}, Val {val}\nPredicted\tExp {np.argmax(preds[0])}, Aro {preds[1]}, Val {preds[2]}')